# Model Evaluation

In [1]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project'

In [4]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
#entity for evaulautiion stage

from pathlib import Path
from dataclasses import dataclass
from typing import List

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: List[int]
    params_batch_size: int


In [9]:
#preparing configguration manager

from src.CNNClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.CNNClassifier.utils import read_yaml, create_directories,save_json

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_validation_data_config(self) ->EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/PetImages",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size =self.params.BATCH_SIZE
        )
        return evaluation_config

In [10]:
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("artifacts/scores.json"), data=scores)

In [11]:
#pipeline

try:
    config = ConfigurationManager()
    evaluation_config = config.get_validation_data_config()
    evalavation =Evaluation(config = evaluation_config)
    evalavation.evaluation()
    evalavation.save_score()
except Exception as e:
    raise e

Found 118 images belonging to 2 classes.


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.5000 - loss: 16.1142
